## Mapping DrugBank small molecule identifiers to other resource identifiers
### Resource link : https://www.drugbank.ca/releases/latest#open-data
### Adapted from https://github.com/dhimmel/drugbank

Using DrugBank as a standard lexicon for small molecule names, I will cross-reference DrugBank identifiers to several other resource identifiers using UniChem and the connectivity information stored within InChiKeys

In [2]:
import os
import csv
import collections
import json
import io
import pandas as pd
import gzip
import glob as glob

import requests

In [2]:
df = pd.read_csv('drugbank_vocabulary.csv')

In [3]:
df.head(10)

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key
0,DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,Hirudin variant-1 | Lepirudin recombinant,NaN
1,DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,Cetuximab | Cétuximab | Cetuximabum | Immunogl...,NaN
2,DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,Deoxyribonuclease (human clone 18-1 protein mo...,NaN
3,DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Interleukin-2/diptheria toxin fus...,NaN
4,DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro...,NaN
5,DB00006,BIOD00076 | BTD00076 | DB02351 | EXPT03302,Bivalirudin,128270-60-0,TN9BEX005G,Bivalirudin | Bivalirudina | Bivalirudinum,OIRCOABEOLEUMC-GEJPAHFPSA-N
6,DB00007,BIOD00009 | BTD00009,Leuprolide,53714-56-0,EFY6W0M8TG,Leuprorelin | Leuprorelina | Leuproreline | Le...,GFIJNRVAKGFPGQ-LIJARHBVSA-N
7,DB00008,BIOD00043 | BTD00043,Peginterferon alfa-2a,198153-51-4,Q46947FE7K,PEG-IFN alfa-2A | PEG-Interferon alfa-2A | Peg...,NaN
8,DB00009,BIOD00050 | BTD00050,Alteplase,105857-23-6,1RXS4UE564,Alteplasa | Alteplase (genetical recombination...,NaN
9,DB00010,BIOD00033 | BTD00033,Sermorelin,86168-78-7,89243S03TE,NaN,NaN


In [4]:
# Drop molecules without an InChI Key
df.dropna(subset = ['Standard InChI Key'], inplace = True)

In [5]:
df.to_csv('drugbank_metadata.tsv', sep = '\t', index = False)

In [6]:
with open('drugbank_metadata.tsv') as read_file:
    reader = csv.DictReader(read_file, delimiter = '\t')
    drugbank = list(reader)

In [7]:
id_to_source = {
    0: None,
    1: 'chembl',
    2: 'drugbank',
    3: 'pdb',
    4: 'iuphar',
    5: 'pubchem_dotf',
    6: 'kegg_ligand',
    7: 'chebi',
    8: 'nih_ncc',
    9: 'zinc',
    10: 'emolecules',
    11: 'ibm',
    12: 'atlas',
    13: 'ibm_patents',
    14: 'fdasrs',
    15: 'surechembl',
    17: 'pharmgkb',
    18: 'hmdb',
    20: 'selleck',
    21: 'pubchem_tpharma',
    22: 'pubchem',
    23: 'mcule',
    24: 'nmrshiftdb2',
    25: 'lincs',
    26: 'actor',
    27: 'recon',
    28: 'molport',
    29: 'nikkaji',
    31: 'bindingdb',
    32: 'comptox',
    33: 'lipidmaps',
    34: 'drugcentral',
    35: 'carotenoiddb',
    36: 'metabolights',
    37: 'brenda',
    38: 'rhea',
    39: 'chemicalbook',
    40: 'dailymed',
    41: 'swisslipids',
    45: 'dailtmed_new',
    46: 'clinicaltrials'
    
}

source_to_id = {v: k for k, v in id_to_source.items()}

In [8]:
def connectivity_query(search_url, target = None, B = 0, C = 0, D = 0, E = 0, F = 0, G = 0):
    """
    https://www.ebi.ac.uk/unichem/info/widesearchInfo
    """
    url = '{search_url}/{A}/{B}/{C}/{D}/{E}/{F}/{G}/{H}'.format(
        search_url = search_url,
        A = source_to_id[target], # Sources
        B = B, # Pattern
        C = C, # Component Mapping
        D = D, # Frequency Block
        E = E, # InChI Length Block
        F = F, # UniChem Labels
        G = G, # Assignment Status
        H = 1, # Data Structure
    )
    response = requests.get(url)
    try:
        response = response.json()
    except ValueError:
        print('cannot decode json:', url)
        return
    if 'error' in response:
        print('UniChem error:', response['error'])
        return
    for assignment in response.values():
        header = assignment.pop(0)
        for match in assignment:
            yield collections.OrderedDict(zip(header, match))

def key_search(inchikey, **kwargs):
    """Search by InChIKeys."""
    if inchikey.startswith('InChIKey='):
        prefix, inchikey = inchikey.split('=', 1)
    base_url = 'https://www.ebi.ac.uk/unichem/rest/key_search'
    search_url = '{base_url}/{StandardInChIKey}'.format(
        base_url = base_url,
        StandardInChIKey = inchikey)
    return connectivity_query(search_url, **kwargs)
    
def cpd_search(source, compound_id, **kwargs):
    """Search by source-specific identifiers."""
    base_url = 'https://www.ebi.ac.uk/unichem/rest/cpd_search'
    search_url = '{base_url}/{src_compound_id}/{src_id}'.format(
        base_url = base_url,
        src_compound_id = compound_id,
        src_id = source_to_id[source])
    return connectivity_query(search_url, **kwargs)

In [9]:
# mapping writer
mapping_path = 'mapping.tsv.gz'
mapping_file = gzip.open(mapping_path, 'wb')
mapping_buffer = io.TextIOWrapper(mapping_file, line_buffering = True)
mapping_fields = ['drugbank_id', 'drugbank_name', 'src_id', 'source_name', 'src_compound_id',
              'C', 'Query_InChIKey', 'CpdId_InChIKey', 'Full_Query_InChI', 'Full_CpdId_InChI',
              'Matching_Query_InChI', 'Matching_CpdId_InChI', 'b', 'i', 'm', 'p', 's', 't']
mapping_writer = csv.DictWriter(mapping_buffer, delimiter = '\t', fieldnames = mapping_fields, extrasaction = 'ignore')
mapping_writer.writeheader()



for drug in drugbank:
    drugbank_id = drug['DrugBank ID']
    drugbank_name = drug['Common name']
    print(drugbank_id, drugbank_name)
    query_matches = list(cpd_search('drugbank', drugbank_id, C = 4))
    if not query_matches:
        print('non-standard InChI: cannot query compound')
        continue
    
    for match in query_matches:
        match['drugbank_id'] = drugbank_id
        match['drugbank_name'] = drugbank_name
        match['source_name'] = id_to_source[int(match['src_id'])]
        mapping_writer.writerow(match)
    
    source_to_matches = dict()
    for match in query_matches:
        match_set = source_to_matches.setdefault(match['source_name'], set())
        match_set.add(match['src_compound_id'])

mapping_file.close()

DB00006 Bivalirudin
DB00007 Leuprolide
DB00014 Goserelin
DB00027 Gramicidin D
DB00035 Desmopressin
DB00050 Cetrorelix
DB00080 Daptomycin
DB00091 Cyclosporine
DB00093 Felypressin
DB00104 Octreotide
DB00106 Abarelix
DB00114 Pyridoxal phosphate
DB00115 Cyanocobalamin
DB00116 Tetrahydrofolic acid
DB00117 Histidine
DB00118 Ademetionine
DB00119 Pyruvic acid
DB00120 Phenylalanine
DB00121 Biotin
DB00122 Choline
DB00123 L-Lysine
DB00125 Arginine
DB00126 Ascorbic acid
DB00127 Spermine
DB00128 Aspartic acid
DB00129 Ornithine
DB00130 L-Glutamine
DB00131 Adenosine phosphate
DB00132 alpha-Linolenic acid
DB00133 Serine
DB00134 Methionine
DB00135 Tyrosine
DB00136 Calcitriol
DB00137 Lutein
DB00138 Cystine
DB00139 Succinic acid
DB00140 Riboflavin
DB00141 N-Acetylglucosamine
DB00142 Glutamic acid
DB00143 Glutathione
DB00144 Phosphatidyl serine
DB00145 Glycine
DB00146 Calcifediol
DB00147 Pyridoxal
DB00148 Creatine
DB00149 Leucine
DB00150 Tryptophan
DB00151 Cysteine
DB00152 Thiamine
DB00153 Ergocalciferol


DB00518 Albendazole
DB00519 Trandolapril
DB00520 Caspofungin
DB00521 Carteolol
DB00522 Bentiromide
DB00523 Alitretinoin
DB00524 Metolazone
DB00525 Tolnaftate
DB00526 Oxaliplatin
DB00527 Cinchocaine
DB00528 Lercanidipine
DB00529 Foscarnet
DB00530 Erlotinib
DB00531 Cyclophosphamide
DB00532 Mephenytoin
DB00533 Rofecoxib
DB00534 Chlormerodrin
DB00535 Cefdinir
DB00536 Guanidine
DB00537 Ciprofloxacin
DB00538 Gadoversetamide
DB00539 Toremifene
DB00540 Nortriptyline
DB00541 Vincristine
DB00542 Benazepril
DB00543 Amoxapine
DB00544 Fluorouracil
DB00545 Pyridostigmine
DB00546 Adinazolam
DB00547 Desoximetasone
DB00548 Azelaic acid
DB00549 Zafirlukast
DB00550 Propylthiouracil
DB00551 Acetohydroxamic acid
DB00552 Pentostatin
DB00553 Methoxsalen
DB00554 Piroxicam
DB00555 Lamotrigine
DB00556 Perflutren
DB00557 Hydroxyzine
DB00558 Zanamivir
DB00559 Bosentan
DB00560 Tigecycline
DB00561 Doxapram
DB00562 Benzthiazide
DB00563 Methotrexate
DB00564 Carbamazepine
DB00565 Cisatracurium
DB00566 Succimer
DB00567

DB00917 Dinoprostone
DB00918 Almotriptan
DB00919 Spectinomycin
DB00920 Ketotifen
DB00921 Buprenorphine
DB00922 Levosimendan
DB00923 Ceforanide
DB00924 Cyclobenzaprine
DB00925 Phenoxybenzamine
DB00926 Etretinate
DB00927 Famotidine
DB00928 Azacitidine
DB00929 Misoprostol
DB00931 Metacycline
DB00932 Tipranavir
DB00933 Mesoridazine
DB00934 Maprotiline
DB00935 Oxymetazoline
DB00936 Salicylic acid
DB00937 Diethylpropion
DB00938 Salmeterol
DB00939 Meclofenamic acid
DB00940 Methantheline
DB00941 Hexafluronium
DB00942 Cycrimine
DB00943 Zalcitabine
DB00944 Demecarium
DB00945 Acetylsalicylic acid
DB00946 Phenprocoumon
DB00947 Fulvestrant
DB00948 Mezlocillin
DB00949 Felbamate
DB00950 Fexofenadine
DB00951 Isoniazid
DB00952 Naratriptan
DB00953 Rizatriptan
DB00954 Dirithromycin
DB00955 Netilmicin
DB00956 Hydrocodone
DB00957 Norgestimate
DB00958 Carboplatin
DB00959 Methylprednisolone
DB00960 Pindolol
DB00961 Mepivacaine
DB00962 Zaleplon
DB00963 Bromfenac
DB00964 Apraclonidine
DB00966 Telmisartan
DB009

DB01378 Magnesium cation
DB01380 Cortisone acetate
DB01382 Glymidine
DB01384 Paramethasone
DB01388 Mibefradil
DB01390 Sodium bicarbonate
DB01392 Yohimbine
DB01393 Bezafibrate
DB01394 Colchicine
DB01395 Drospirenone
DB01396 Digitoxin
DB01397 Magnesium salicylate
DB01399 Salsalate
DB01400 Neostigmine
DB01401 Choline magnesium trisalicylate
DB01403 Methotrimeprazine
DB01405 Temafloxacin
DB01406 Danazol
DB01407 Clenbuterol
DB01408 Bambuterol
DB01409 Tiotropium
DB01410 Ciclesonide
DB01411 Pranlukast
DB01412 Theobromine
DB01413 Cefepime
DB01414 Cefacetrile
DB01415 Ceftibuten
DB01416 Cefpodoxime
DB01418 Acenocoumarol
DB01419 Antrafenine
DB01420 Testosterone propionate
DB01421 Paromomycin
DB01422 Nitroxoline
DB01423 Stepronin
DB01424 Aminophenazone
DB01425 Alizapride
DB01426 Ajmaline
DB01427 Amrinone
DB01428 Oxybenzone
DB01429 Aprindine
DB01430 Almitrine
DB01431 Allylestrenol
DB01433 Methadyl acetate
DB01434 19-norandrostenedione
DB01435 Antipyrine
DB01436 Alfacalcidol
DB01437 Glutethimide
DB0

DB01713 UDP-alpha-D-xylose
DB01714 N-Methyl-Lysine
DB01715 7,8-Diamino-Nonanoic Acid
DB01716 2-propenyl-N-acetyl-neuramic acid
DB01717 Bis(Adenosine)-5'-Pentaphosphate
DB01718 Cetrimonium
DB01719 Thio-Maltopentaose
DB01720 (2z)-2-(Benzoylamino)-3-[4-(2-Bromophenoxy)Phenyl]-2-Propenoic Acid
DB01721 N-[2-hydroxy-1-indanyl]-5-[(2-tertiarybutylaminocarbonyl)-4(benzo[1,3]dioxol-5-ylmethyl)-piperazino]-4-hydroxy-2-(1-phenylethyl)-pentanamide
DB01723 {3-[3-(3,4-Dimethoxy-Phenyl)-1-(1-{1-[2-(3,4,5-Trimethoxy-Phenyl)-Butyryl]-Piperidin-2yl}-Vinyloxy)-Propyl]-Phenoxy}-Acetic Acid
DB01724 Reduced Threonine
DB01725 2-{2-hydroxy-[1,1'-biphenyl]-3-yl}-1H-1,3-benzodiazole-5-carboximidamide
DB01726 2-Aminophenol
DB01727 Isocitric Acid
DB01728 3-[Aminoethylphosphoryl]-[1,2-Di-Palmitoyl]-Sn-Glycerol
DB01729 (1s,3s,4s)-1,3,4-Triphospho-Myo-Inositol
DB01731 (S)-wiskostatin
DB01732 (4r,5s,6s,7r)-1,3-Dibenzyl-4,7-Bis(Phenoxymethyl)-5,6-Dihydroxy-1,3 Diazepan-2-One
DB01733 L-Phospholactate
DB01734 3-(Oxalyl-

...

DB12888 Ezutromid
DB12889 Velneperit
DB12890 Dihydrexidine
DB12892 MGB-BP-3
DB12894 Raluridine
DB12895 TD-139
DB12896 PSI-352938
DB12897 MK-7622
DB12899 TT-301
DB12900 Irdabisant
DB12901 Fiacitabine
DB12902 Trofosfamide
DB12903 DEBIO-1347
DB12904 ZSTK-474
DB12905 Samarium Sm-153
DB12906 LY-2334737
DB12907 Mannose
DB12910 Emicerfont
DB12911 Nicoboxil
DB12912 Nolatrexed
DB12914 MGL-3196
DB12916 Mitolactol
DB12919 T-62
DB12920 Pinometostat
DB12921 Chlorsulfaquinoxaline
DB12923 Gallopamil
DB12924 Ozenoxacin
DB12925 Crolibulin
DB12926 Cafedrine
DB12927 Theodrenaline
DB12928 GET-73
DB12929 JNJ-39220675
DB12930 Opipramol
DB12931 Fenobam
DB12932 Merotocin
DB12933 RO-4987655
DB12934 Granotapide
DB12935 Remogliflozin etabonate
DB12937 Quercetin-3'-O-phosphate
DB12938 Isoxaflutole
DB12939 Nikkomycin Z
DB12940 Helium He-3
DB12941 Darolutamide
DB12942 Lactitol
DB12944 Phloroglucinol
DB12945 Dihydralazine
DB12946 8-cyclopentyl-1,3-dipropylxanthine
DB12947 Doxifluridine
DB12948 Didox
DB12949 PF-03382

DB13364 Feprazone
DB13366 Hydrochloric acid
DB13367 Cloricromen
DB13368 Motretinide
DB13369 Benzilone
DB13370 Bromisoval
DB13371 Difenpiramide
DB13374 Vincamine
DB13376 Succinimide
DB13377 Vinbarbital
DB13378 Norfenefrine
DB13379 Chiniofon
DB13380 Difemerine
DB13381 Sodium feredetate
DB13382 Chlorproethazine
DB13383 Diiodohydroxypropane
DB13384 Melitracen
DB13386 Epimestrol
DB13387 Hematin
DB13388 Phenylmercuric nitrate
DB13389 Iopydol
DB13391 Levoverbenone
DB13392 Mercurochrome
DB13393 Emetine
DB13394 Adrenalone
DB13395 Eprozinol
DB13396 Neocitrullamon
DB13397 Prenoxdiazine
DB13398 Oxyfedrine
DB13399 Terguride
DB13400 Linsidomine
DB13401 Metildigoxin
DB13402 Acetylglycinamide chloral hydrate
DB13403 Oxypertine
DB13404 Edetate sodium
DB13405 Mefruside
DB13406 Carbutamide
DB13407 Nifenazone
DB13408 Methylpropylpropanediol dinitrate
DB13409 Rokitamycin
DB13410 Guanoxabenz
DB13411 Lofepramine
DB13412 Pirisudanol
DB13413 Phenglutarimide
DB13414 Fenyramidol
DB13415 Pentetrazol
DB13416 Sulbu

DB13776 Ibacitabine
DB13777 Prenalterol
DB13778 Cefazedone
DB13779 Guanoclor
DB13780 Aluminium clofibrate
DB13781 Xamoterol
DB13782 Imipramine oxide
DB13783 Acemetacin
DB13784 Dixyrazine
DB13785 Dropropizine
DB13786 Magnesium orotate
DB13787 Tilidine
DB13788 Chlorbenzoxamine
DB13789 Aluminium acetotartrate
DB13790 Fipexide
DB13791 Penfluridol
DB13792 Clopamide
DB13793 Vinburnine
DB13794 Dimethoxanate
DB13795 Brodimoprim
DB13796 Dibunate
DB13797 Iodocholesterol (131I)
DB13798 Demoxytocin
DB13799 Ethadione
DB13800 Calcium levulinate
DB13801 Muzolimine
DB13802 Epomediol
DB13803 Xipamide
DB13804 Benzylthiouracil
DB13805 Reposal
DB13806 Linopirdine
DB13807 Tisopurine
DB13808 Mebhydrolin
DB13810 Dimemorfan
DB13812 Bufylline
DB13813 Iodoform
DB13814 Talampicillin
DB13815 Xenysalate
DB13816 Aspoxicillin
DB13817 Carbromal
DB13818 Bibrocathol
DB13819 Proxazole
DB13820 Oxomemazine
DB13821 Ceftezole
DB13822 Meprotixol
DB13823 Pipemidic acid
DB13824 Enprostil
DB13825 Fedrilate
DB13826 Etohexadiol
D

DB14487 Zinc acetate
DB14488 Ferrous gluconate
DB14489 Ferrous succinate
DB14490 Ferrous ascorbate
DB14491 Ferrous fumarate
DB14492 Potassium triiodide
DB14493 Zinc glycinate
DB14494 Zinc carbonate
DB14495 Manganese citrate
DB14496 Sodium molybdate
DB14497 Calcium magnesium potassium carbonate chloride hydroxide
DB14498 Potassium acetate
DB14499 Potassium sulfate
DB14500 Potassium
DB14501 Ferrous glycine sulfate
DB14502 Sodium phosphate, dibasic
DB14505 Sodium borate
DB14506 Lithium hydroxide
DB14507 Lithium citrate
DB14508 Lithium succinate
DB14509 Lithium carbonate
DB14510 Ferrous cation
DB14511 Acetate
DB14512 Mometasone furoate
DB14513 Magnesium
DB14514 Magnesium levulinate
DB14515 Magnesium lactate
DB14516 Sodium cation
DB14517 Aluminium phosphate
DB14518 Aluminum acetate
DB14519 Aluminum cation
DB14520 Tetraferric tricitrate decahydrate
DB14521 Silver cation
DB14522 Hydroxide ion
DB14523 Phosphate ion
DB14524 Gallium cation
DB14525 Chromic cation
DB14526 Chromic citrate
DB14527 C

DB15023 BMS-791826
DB15024 Apararenone
DB15026 CXA-10
DB15027 Adriforant
DB15028 MK-1064
DB15029 AZD-8186
DB15031 Nemorexant
DB15033 Flortaucipir
DB15034 PRI-724
DB15035 Zanubrutinib
DB15036 Sitravatinib
DB15038 Litoxetine
DB15039 ITI-214
DB15040 TP-271
DB15041 Florilglutamic acid F-18
DB15046 LY-2881835
DB15047 BMS-919373
DB15048 Licogliflozin
DB15050 Neopterin
DB15051 Ethaselen
DB15052 Ansofaxine
DB15054 TAS-117
DB15055 ABT-639
DB15056 Bifenthrin
DB15057 NUC-1031
DB15058 Flutemetamol
DB15059 Aprocitentan
DB15062 Inarigivir
DB15063 Inarigivir soproxil
DB15065 PF-06291874
DB15068 Agerafenib
DB15071 Omidenepag isopropyl
DB15073 Ribose
DB15075 4-demethyl-4-cholesteryloxycarbonylpenclomedine
DB15078 Rogaratinib
DB15079 Troriluzole
DB15080 ELX-02
DB15084 GS-9191
DB15085 Fulacimstat
DB15086 Ocrylate
DB15087 Psicose
DB15091 Upadacitinib
DB15092 LY-2624803
DB15096 18-methoxycoronaridine
DB15097 Gefapixant
DB15098 Morinidazole
DB15099 GW-468816
DB15102 Pemigatinib
DB15105 PF-06751979
DB15106 S

In [9]:
# write source-specific mapping files
mapping_path = 'mapping.tsv.gz'
mapping_file = gzip.open(mapping_path, 'rb')
mapping_buffer = io.TextIOWrapper(mapping_file)
reader = csv.DictReader(mapping_buffer, delimiter='\t')
source_to_pairs = dict()
for row in reader:
    pair = row['drugbank_id'], row['src_compound_id']
    pairs = source_to_pairs.setdefault(row['source_name'], set())
    pairs.add(pair)
mapping_file.close()

In [15]:
# write source-specific mapping files
mapping_path = 'mapping.tsv.gz'
mapping_file = gzip.open(mapping_path, 'rb')
mapping_buffer = io.TextIOWrapper(mapping_file)
reader = csv.DictReader(mapping_buffer, delimiter='\t')
source_to_pairs = dict()
for row in reader:
    pair = row['drugbank_id'], row['src_compound_id'], row['Query_InChIKey']
    pairs = source_to_pairs.setdefault(row['source_name'], set())
    pairs.add(pair)
mapping_file.close()

del source_to_pairs['drugbank']
for source, pairs in source_to_pairs.items():
    path = os.path.join('mapping_files/{}.tsv'.format(source))
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    write_file = open(path, 'w')
    writer = csv.writer(write_file, delimiter='\t')
    writer.writerow(['drugbank_id', '{}_id'.format(source),'inchi_key'])
    writer.writerows(sorted(pairs))
    write_file.close()

In [5]:
# importing df of drugbank IDs matched to inchi keys
df_metadata = pd.read_csv('drugbank_metadata.tsv', sep = '\t',  usecols = ['DrugBank ID','Standard InChI Key'])
df_metadata = df_metadata.rename(columns = {'DrugBank ID':'drugbank_id'})

# overwriting mapping files with updated inchi_key from DrugBank
import glob as glob
path = "mapping_files/*.tsv"
for filename in glob.iglob(path):
    drugbank_mapping = pd.read_csv(filename, sep = '\t')
    df_out = drugbank_mapping.merge(df_metadata)
    df_out = df_out.drop(columns = ['inchi_key'])
    df_out = df_out.rename(columns = {'Standard InChI Key': 'inchi_key'})
    df_out.to_csv(filename, sep = '\t', index = False)